# Performance Testing

In [1]:
import json

class ConfigReader:
    def get_value(self, key):
        with open('config.json') as json_data_file:
            data = json.load(json_data_file)
        return data[key]

In [6]:
# TODO: change to use any sql db, use sqlite3 for now.
import sqlite3
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String
import urllib
from pandas.io import sql
from datetime import datetime

class DbManager:
    def __init__(self):
        reader = ConfigReader()
        db_connection = reader.get_value("db_connection")        
        self.conn_string = '{db_engine}+{connector}://{user}:{password}@{server}/{database}'.format(
            db_engine=db_connection['db_engine'],
            connector=db_connection['connector'],
            user=db_connection['user'],
            password=db_connection['password'],
            server=db_connection['server'],
            database=db_connection['database'])
 
    def get_ratings(self):
        return sql.read_sql("SELECT userId, itemId, rating, timestamp FROM ratings;", create_engine(self.conn_string))

    def get_movies(self, term):
        return sql.read_sql("SELECT movieId, title, genres FROM movies WHERE title LIKE '%{term}%';".format(term=term), create_engine(self.conn_string))

    def get_movie(self, id):
        return sql.read_sql("SELECT title, genres FROM movies WHERE movieId = {id};".format(id=id), create_engine(self.conn_string))

    def get_links(self):
        return sql.read_sql("SELECT movieId, imdbId, tmdbId FROM links;", create_engine(self.conn_string))

    def get_users(self):
        return sql.read_sql("SELECT distinct userId FROM ratings;", create_engine(self.conn_string))
    
    def insert_rating(self, user_id, movie_id, rating_value):        
        sql.execute("INSERT INTO ratings VALUES ({userId}, {itemId}, {rating}, '{timestamp}')".format(
                    userId=user_id,
                    itemId=movie_id,
                    rating=rating_value,
                    timestamp=datetime.timestamp(datetime.now()),
                    ), create_engine(self.conn_string))

    def remove_ratings(self, user_id):
        sql.execute("DELETE FROM ratings WHERE userId = {userId}".format(
                    userId=user_id), create_engine(self.conn_string))

    def insert_and_get_min_user_id(self):
        db_list = sql.read_sql("SELECT MIN(userId) as userId FROM users;", create_engine(self.conn_string))
        #min_user_id = [m[1]['userId'] for m in db_list.iterrows()][0]
        min_user_id = db_list.iloc[0]['userId']
        user_id = int(min_user_id - 1)
        sql.execute("INSERT INTO users(userId) VALUES ({userId})".format(userId=user_id), create_engine(self.conn_string))
        return user_id
    

## Get random users

In [9]:
dbManager = DbManager()
db_users = dbManager.get_users()
n_users = len(db_users)
n_users

162543

In [42]:
n_rand_users = 100
n_rand_users = db_users.sample(n=n_rand_users)
n_rand_users

,userId
40951,40950
19730,19729
97773,97772
87733,87732
21472,21471
...,...
88566,88565
124903,124902
35077,35076
17605,17604


## Request recommendations for random users

In [ ]:
import requests
import time
from datetime import datetime

In [22]:
base_url = 'http://127.0.0.1:8000'

In [23]:
  def get_recs_from_recserver(users, nr_recs, algo, items):
        is_a_rec_request = True if algo == 'popular' or algo == 'topn' else False
        recs = []
        for userId in users:
            if is_a_rec_request:
                url = f'{base_url}/algorithms/{algo}/recommendations?user_id={userId}&num_recs={nr_recs}'
            else:
                url = f'{base_url}/algorithms/{algo}/predictions?user_id={userId}&items={items}'
            r = requests.get(url)
            data = r.json()
            recs.append({'user': userId, 'recs': data['recommendations'] if is_a_rec_request else data['predictions']})
        return recs

In [49]:
algo = 'popular'
n_recs = 5

In [50]:
#t = time.process_time()
start = datetime.now()
for idx, row in n_rand_users.iterrows():
    get_recs_from_recserver([row['userId']], n_recs, algo, None)
#elapsed_time = time.process_time() - t
time_taken = datetime.now() - start
#print(f'cpu time: {elapsed_time}')
print(f'time taken: {time_taken}')

time taken: 0:00:08.093078
